In [1]:
from jarvis.core.atoms import Atoms
from jarvis.io.vasp.inputs import Poscar
from jarvis.db.figshare import data
import os
import pandas as pd
import json


In [ ]:
#Structure file generator for running ALIGNN scratch model
# struc_path = "/path/to/alignn/root_dir"
struc_path = "./"
dft_3d = data(dataset='dft_3d')
jids_file = []
jids = []
for i in dft_3d:
    atoms = Atoms.from_dict(i['atoms'])
    poscar = Poscar(atoms)
    jid = i['jid']
    jids.append(jid)
    jids_file.append(jid+'.vasp')
    filename1 = os.path.join(struc_path, 'POSCAR-'+jid+'.vasp')
    filename2 = os.path.join(struc_path, jid+'.vasp')
    if (not os.path.exists(filename1)) and (not os.path.exists(filename2)):
        poscar.write_file(filename2)
data = {'jid_file': jids_file, 'jid': jids}
df = pd.DataFrame(data)
csv_path = os.path.join(struc_path, "id_prop_full.csv")
df.to_csv(csv_path, header=False, index=False)
#save id_prop csv



In [3]:
#id prop csv (for running ALIGNN) based on the data split
# props =  ["slme", "spillage", "magmom_outcar", "mbj_bandgap", "Tc_supercon"]
split_json_dir = "../data/split/"
# struc_path = "/path/to/alignn/root_dir"
struc_path = "./"
dft_3d = data(dataset='dft_3d')

prop = "mbj_bandgap"
# df_file = pd.read_csv(full_data_path, header=None, names = ["file", "jid"])
split_json_path = os.path.join(split_json_dir, f"dataset_split_{prop}.json")
prop_folder = struc_path
os.makedirs(prop_folder , exist_ok=True)
with open(split_json_path, 'r') as file:
    # Load the JSON data into a dictionary
    ids_dict = json.load(file)
    print(prop)
    ids_train = [id + '.vasp' for id in ids_dict['id_train']]
    print(len(ids_train))
    ids_val = [id + '.vasp' for id in ids_dict['id_val']]
    print(len(ids_val))
    ids_test = [id + '.vasp' for id in ids_dict['id_test']]
    print(len(ids_test))
jids_file = []
values = []
for i in dft_3d:
    if i[prop]!='na':
        atoms = Atoms.from_dict(i['atoms'])
        poscar = Poscar(atoms)
        jid = i['jid']
        value = i[prop]
        values.append(value)
        jids_file.append(jid+'.vasp')

data_id = {'jid_file': jids_file, 'prop': values}
df = pd.DataFrame(data_id)
df_train = df[df.jid_file.isin(ids_train)].copy()
df_val = df[df.jid_file.isin(ids_val)].copy()
df_test = df[df.jid_file.isin(ids_test)].copy()
df_sorted = pd.concat([df_train, df_val, df_test])

csv_path = os.path.join(prop_folder, "id_prop.csv")

df_sorted.to_csv(csv_path, header=False, index=False)



Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
mbj_bandgap
15642
1957
1957
